# 1- Imports

In [2]:
## Essenciais
import pandas as pd
import numpy  as np

import pickle as pkl

### ML
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics      import f1_score  

# 2- Importação dos dados

In [4]:
xtr = pkl.load( open( '../data/pickle/xtr.sav', 'rb' ) )
xte = pkl.load( open( '../data/pickle/xte.sav', 'rb' ) )

ytr = pkl.load( open( '../data/pickle/ytr.sav', 'rb' ) )
yte = pkl.load( open( '../data/pickle/yte.sav', 'rb' ) )